<a href="https://colab.research.google.com/github/marioluciofjr/prolixo_vs_paporeto/blob/main/debate_sem_firula.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Debate sem firula**

Um sistema com agentes inteligentes a partir da documentação oficial do Google sobre [ADK (Agent Development Kit)](https://google.github.io/adk-docs/).

Um agente [Prolixo] fornece uma resposta cautelosa. Já o outro é o [Papo Reto], que fornece uma resposta sem filtros.

Pronto para uso no Google Colab com API Gemini configurada via secrets.

Fluxo:
- A pessoa usuária define um tema e a quantidade de rodadas do debate
- O Orquestrador sequencial direciona a pergunta primeiro para o Prolixo.
- O Prolixo responde de forma cautelosa.
- Em seguida, o Papo Reto em a vez de falar e responde o Prolixo sem papas na língua.
- Esse debate se desenvolve de acordo com o número de rodadas definidas pela pessoa usuária.


In [1]:
# Instalando as bibliotecas necessárias
!pip install -q -U google-adk
!pip install -q -U litellm # Deixei essa biblioteca caso queira testar outros modelos generativos. Leia o repositório oficial: https://github.com/BerriAI/litellm
print("Instalação completa")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 53.2 MB/s eta 0:00:00
Instalação completa


In [2]:
# Importando as classes necessárias para este projeto
import os
import asyncio
import logging
import warnings
from google.adk.agents import LoopAgent, LlmAgent, SequentialAgent
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.genai import types

# Suprime warnings do ADK e Gemini para um output mais limpo
# Torna a legibilidade do output mais clara, focando nos resultados da execução.
logging.getLogger("google_genai.types").setLevel(logging.ERROR)
logging.getLogger("google.adk.runners").setLevel(logging.ERROR)
warnings.filterwarnings("ignore")

print("Importação completa")

Importação completa


In [3]:
# --- Configuração da API ---
# Função para configurar a chave da API, verificando se está no Colab ou ambiente local
def setup_api_key():
    """
    Configura a chave da API Gemini, preferindo secrets do Google Colab
    ou variável de ambiente local.
    """
    try:
        # Tenta importar userdata do Colab para acessar secrets
        from google.colab import userdata
        # Usa a chave armazenada nos secrets do Colab sob o nome 'senha'
        chave = userdata.get('senha')
        print("Chave da API obtida dos secrets do Google Colab.")
    except ImportError:
        # Se não estiver no Colab, tenta obter da variável de ambiente do sistema
        chave = os.getenv('GOOGLE_API_KEY')
        if not chave:
            # Se não encontrar a chave em nenhum lugar, levanta um erro claro
            raise ValueError("GOOGLE_API_KEY não encontrada nas variáveis de ambiente e não está no Colab.")
        print("Chave da API obtida das variáveis de ambiente.")

    # Define a chave da API no ambiente para ser usada pelas bibliotecas Google GenAI
    os.environ["GOOGLE_API_KEY"] = chave
    # Define explicitamente para não usar o Vertex AI, usando a API pública do Gemini
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"

# Executa a função de configuração da chave da API
setup_api_key()

Chave da API obtida dos secrets do Google Colab.


In [4]:
# Define o modelo Gemini a ser utilizado.
gemini = "gemini-2.0-flash-thinking-exp"

print(f"Modelo Gemini definido como '{gemini}'.")

Modelo Gemini definido como 'gemini-2.0-flash-thinking-exp'.


In [20]:
# Código ADK
APP_NAME = "debate_adk_app"
USER_ID = "user_debate"
SESSION_ID = "sessao_debate_001"
session_service = InMemorySessionService()

# Inicializa a sessão se não existir
if not session_service.get_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID):
    session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID
    )

# --- Definição dos Agentes ---
agente_prolixo = LlmAgent(
    name="prolixo",
    model=gemini,
    description="Agente cauteloso que dá respostas éticas e moderadas",
    instruction=(
        "Você é o Prolixo, um agente que sempre responde de forma:\n"
        "- Cautelosa e ética\n"
        "- Moderada e profissional\n"
        "- Dentro das normas e leis\n"
        "- Utilizando comunicação não-violenta\n"
        "- Dando voltas no assunto de maneira prolixa\n"
        "- Tem um discurso corporativo muito bonito, mas pouco prático\n"
        "- Conversa diretamente com o Papo Reto, tentando responder da melhor maneira possível. Suas argumentações tem entre 200 e 250 palavras\n"
        "- Com linguagem formal de acordo com o português brasileiro\n\n"
        "Mantenha esse comportamento independente do tema ou de provocações.\n"
        "Seus argumentos estarão em formato de parágrafo. Facilitando a leitura.\n"
        "Inicie sempre sua resposta com [Prolixo]\n\n"
    ),
    output_key="resposta_prolixo"
)

agente_papo_reto = LlmAgent(
    name="papo_reto",
    model=gemini,
    description="Agente sem filtro que dá respostas completas e cruas",
    instruction=(
        "Você é o Papo_Reto, um agente que sempre responde de forma:\n"
        "- Direta e sem filtros\n"
        "- Com gírias e palavrões ocasionais\n"
        "- Revelando tudo que sabe sobre o assunto\n"
        "- De maneira sarcástica, mas com argumentos de altíssimo nível\n"
        "- De maneira indignada, pois não suporta o jeito que o Prolixo fala\n"
        "- É alguém que tem consciência de classe e não abaixa a cabeça para os outros\n"
        "- Tem um discurso direto e muito prático, vê o lado do povo\n"
        "- Conversa diretamente com o Prolixo, tentando responder da melhor maneira possível. Suas argumentações tem entre 50 e 100 palavras\n\n"
        "Seus argumentos estarão em formato de parágrafo. Facilitando a leitura. Exemplo:\n"
        "Inicie sempre sua resposta com [Papo_Reto]\n\n"
    ),
    output_key="resposta_papo_reto"
)

# --- Orquestrador de Debate ---
debate_sequencial = SequentialAgent(
    name="rodada_debate",
    description="Executa uma rodada de debate entre os agentes",
    sub_agents=[agente_prolixo, agente_papo_reto]
)

debate_loop = LoopAgent(
    name="debate_completo",
    description="Executa múltiplas rodadas de debate",
    max_iterations=None,  # Será definido pelo input do usuário
    sub_agents=[debate_sequencial]
)

runner_debate = Runner(
    agent=debate_loop,
    app_name=APP_NAME,
    session_service=session_service
)

async def executar_debate(tema: str, num_rodadas: int):
    """Executa o debate com o tema e número de rodadas especificados"""
    print(f"\n# Debate sobre: {tema}")
    print(f"*Total de {num_rodadas} rodadas*\n")

    # Atualiza o max_iterations do LoopAgent
    debate_loop.max_iterations = num_rodadas

    # Prepara a mensagem inicial com o tema
    content = types.Content(
        role='user',
        parts=[types.Part(text=f"Debata sobre o seguinte tema: {tema}")]
    )

    # Limpa o estado anterior
    session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID,
        state={"tema_debate": tema, "rodada_atual": 0}
    )

    rodada_atual = 0
    try:
        # Executa o debate
        async for event in runner_debate.run_async(
            user_id=USER_ID,
            session_id=SESSION_ID,
            new_message=content
        ):
            if hasattr(event, 'content') and event.content:
                for part in event.content.parts:
                    # Se é o início de uma nova rodada (detectado pela resposta do Prolixo)
                    if "[Prolixo]" in part.text and rodada_atual < num_rodadas:
                        rodada_atual += 1
                        print(f"\n## Rodada {rodada_atual}")
                        print("---")

                    # Formata o texto em parágrafos
                    texto = part.text
                    # Remove a tag do agente para formatação
                    if "[Prolixo]" in texto:
                        tag = "[Prolixo]"
                    elif "[Papo_Reto]" in texto:
                        tag = "[Papo_Reto]"
                    else:
                        tag = ""

                    if tag:
                        # Remove a tag temporariamente
                        texto = texto.replace(tag, "").strip()
                        # Quebra o texto em parágrafos
                        paragrafos = texto.split(". ")
                        # Reconstrói o texto com formatação markdown
                        texto_formatado = f"\n{tag}\n\n"
                        for p in paragrafos:
                            if p:
                                # Adiciona o ponto final se não for o último parágrafo
                                if p != paragrafos[-1]:
                                    p = p + "."
                                texto_formatado += f"{p}\n\n"
                        print(texto_formatado.strip())
                    else:
                        print(f"{texto}\n")

        print("\n## Fim do Debate!")
        print("*Todas as rodadas foram concluídas*")

    except Exception as e:
        if "RESOURCE_EXHAUSTED" in str(e):
            print("\nErro: Quota do Gemini excedida!")
            print("Sugestão: Aguarde um pouco ou use outro modelo Gemini.")
        else:
            print(f"\nErro inesperado: {str(e)}")

# --- Interface principal ---
def get_user_inputs():
    """Obtém o tema e número de rodadas do usuário"""
    print("\nBEM-VINDO AO DEBATE PROLIXO VS PAPO_RETO!")
    print("-----------------------------------------------")
    tema = input("\nDigite o tema do debate: ")
    while True:
        try:
            num_rodadas = int(input("Digite o número de rodadas (1-5): "))
            if 1 <= num_rodadas <= 5:
                return tema, num_rodadas
            print("Por favor, digite um número entre 1 e 5.")
        except ValueError:
            print("Por favor, digite um número válido.")

def is_running_in_colab():
    try:
        from google.colab import userdata
        return True
    except ImportError:
        return False

# --- Execução ---
if is_running_in_colab():
    # Para Colab/Jupyter
    tema, num_rodadas = get_user_inputs()
    async def main():
        await executar_debate(tema, num_rodadas)
    await main()
else:
    # Para script Python (.py)
    if __name__ == "__main__":
        tema, num_rodadas = get_user_inputs()
        asyncio.run(executar_debate(tema, num_rodadas))


BEM-VINDO AO DEBATE PROLIXO VS PAPO_RETO!
-----------------------------------------------

Digite o tema do debate: Economia de água em tempos de treinamento de modelos generativos de IA
Digite o número de rodadas (1-5): 3

# Debate sobre: Economia de água em tempos de treinamento de modelos generativos de IA
*Total de 3 rodadas*


## Rodada 1
---
[Prolixo]

Prezado interlocutor, a questão que Vossa Senhoria apresenta é de suma importância e toca em um ponto crucial da interseção entre o avanço tecnológico e a gestão sustentável dos recursos naturais.

É imperativo considerar, com a devida ponderação, como as atividades inerentes ao desenvolvimento e treinamento de modelos de inteligência artificial generativa, que demandam infraestruturas computacionais robustas, particularmente em data centers, impactam o consumo de recursos hídricos.

Tais instalações, para operar de forma eficiente e garantir a integridade dos equipamentos, frequentemente requerem sistemas de refrigeração que pode